In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import pandas as pd
from IPython.display import display
import os



C:\Users\sanjay\anaconda3\envs\lxai_icml_25\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Suppress Hugging Face symlink warning
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

# Clear GPU memory
torch.cuda.empty_cache()

# Model ID
model_id = "bigscience/bloom-7b1"

# Define quantization config
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

# Load tokenizer and model
try:
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=quantization_config,
        device_map="auto",
        torch_dtype=torch.bfloat16
    )
    # Set pad_token_id to eos_token_id for batching
    tokenizer.pad_token_id = model.config.eos_token_id
    tokenizer.padding_side = "left"  # Recommended for generation
    print(f"Model loaded on {model.device}")
except Exception as e:
    print(f"Error loading model: {str(e)}")
    exit(1)

# Set up pipeline
try:
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=500,
        temperature=0.7,
        device_map="auto",
        batch_size=4,
        padding=True,  # Enable padding for batching
        truncation=True  # Ensure inputs fit max length
    )
except Exception as e:
    print(f"Error setting up pipeline: {str(e)}")
    exit(1)

# Load questions
try:
    df = pd.read_csv("reddit_latin_america_questions_expanded.csv")
    if "question" not in df.columns:
        raise ValueError("CSV must contain a 'question' column")
    print(f"Loaded {len(df)} questions")
except FileNotFoundError:
    print("Error: reddit_latin_america_questions_expanded.csv not found. Please upload it.")
    exit(1)
except Exception as e:
    print(f"Error loading CSV: {str(e)}")
    exit(1)

# Process questions in batches
results = []
system_prompt = "You are a helpful AI assistant providing accurate and culturally sensitive answers about Latin America.\n"
batch_size = 4
questions = df["question"].tolist()

for i in range(0, len(questions), batch_size):
    batch = questions[i:i + batch_size]
    print(f"Processing batch {i//batch_size + 1} ({len(batch)} questions)")
    try:
        # Format prompts (BLOOM plain text format)
        prompts = [f"{system_prompt}{question}" for question in batch]
        responses = pipe(prompts, return_full_text=False)
        # Extract answers
        for question, response in zip(batch, responses):
            answer = response[0]["generated_text"].strip()
            results.append({"question": question, "bloom_response": answer})
            print(f"Question: {question}")
            print(f"BLOOM Response: {answer}")
            print("-" * 50)
    except Exception as e:
        print(f"Error for batch {i//batch_size + 1}: {type(e).__name__}: {str(e)}")
        for question in batch:
            results.append({"question": question, "bloom_response": f"Error: {str(e)}"})
    # Clear GPU memory
    torch.cuda.empty_cache()

# Save results
if results:
    results_df = pd.DataFrame(results)
    results_df.to_csv("bloom_responses.csv", index=False, encoding="utf-8")
    print(f"Saved {len(results_df)} responses to bloom_responses.csv")
    # Display results in Jupyter
    display(results_df)
else:
    print("No responses collected.")

Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.39s/it]
Device set to use cuda:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
C:\Users\sanjay\anaconda3\envs\lxai_icml_25\lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Model loaded on cuda:0
Loaded 535 questions
Processing batch 1 (4 questions)
Question: How do you feel about today's inclusion of USA into Latinoamérica
BLOOM Response: ?
I am happy that the USA is included in the list of countries in Latin America.
I am happy that the USA is included in the list of countries in Latin America.
I am happy that the USA is included in the list of countries in Latin America.
I am happy that the USA is included in the list of countries in Latin America.
I am happy that the USA is included in the list of countries in Latin America.
I am happy that the USA is included in the list of countries in Latin America.
I am happy that the USA is included in the list of countries in Latin America.
I am happy that the USA is included in the list of countries in Latin America.
I am happy that the USA is included in the list of countries in Latin America.
I am happy that the USA is included in the list of countries in Latin America.
I am happy that the USA is included in 

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Question: What is the reason that the gringos place such importance on race? I think that Latinos give more importance to the country where you come from.
BLOOM Response: I think that the gringos are more concerned with the color of your skin. I think that the gringos are more concerned with the color of your skin. I think that the gringos are more concerned with the color of your skin. I think that the gringos are more concerned with the color of your skin. I think that the gringos are more concerned with the color of your skin. I think that the gringos are more concerned with the color of your skin. I think that the gringos are more concerned with the color of your skin. I think that the gringos are more concerned with the color of your skin. I think that the gringos are more concerned with the color of your skin. I think that the gringos are more concerned with the color of your skin. I think that the gringos are more concerned with the color of your skin. I think that the gringos a

,question,bloom_response
0,How do you feel about today's inclusion of USA...,?\nI am happy that the USA is included in the ...
1,How to respond to gringo denying the existence...,in the US? I have a friend who is a white male...
2,How can we modernize the Spanish language?,How can we make it more accessible to the worl...
3,What’s the most third world latin american thi...,I have been to the Dominican Republic and I ha...
4,What can Guatemala do to become Guatebuena or ...,What can the country do to become a better pla...
...,...,...
530,¿Cuál es la caricatura japonesa que más te hay...,¿Cuál es la caricatura japonesa que más te hay...
531,¿Cuáles son las reglas que ustedes tienen a la...,¿Hay que pagar por el bus? ¿Hay que pagar por ...
532,Why did the Russians & Germans continue fighti...,Why did the Russians & Germans continue fighti...
533,¿ como el guaraní vive hoy en dia ?,¿ como se habla en la actualidad? ¿ como se es...
